In [11]:
import pandas as pd
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import json

In [22]:
df = pd.read_json('./final_data_records_melanie.json')
cleaned_df = df.drop_duplicates(subset=['spotify_track_uri'])
print(df.shape[0])
print(cleaned_df.shape[0])

21409
4284


In [26]:
# Load environment variables from .env file
load_dotenv()

# Access environment variables
my_client_id = os.getenv('CLIENT_ID')
my_client_secret = os.getenv('CLIENT_SECRET')

client_credentials_manager = SpotifyClientCredentials(client_id=my_client_id,
                                                      client_secret=my_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [27]:
album_dict = {}
resume_index = 0

In [28]:
for index, row in cleaned_df.iterrows():
    split = row["spotify_track_uri"].split(":")
    spotify_uri = split[2]

    try:
        # more info about audio features: https://developer.spotify.com/documentation/web-api/reference/get-audio-features
        track_info = sp.track(spotify_uri)

        if (track_info == None):
            print("No track info found for ", row["master_metadata_track_name"])
            continue
        else:
            try:
                album_cover_url = track_info['album']['images'][0]['url']
                album_dict[row["spotify_track_uri"]] = album_cover_url
            except:
                print("No album cover found for", row["master_metadata_track_name"], "with spotify uri:", row["spotify_track_uri"])
                continue
            
    except spotipy.SpotifyException as e:
        # Rate limiting error
        if e.http_status == 429:
            print("Rate limited.")
            print(e)
            resume_index = index
            break

No album cover found for  Live Without Love with spotify uri:  spotify:track:3G5d8eTsM2GY1UTaEYwpOj


In [29]:
file_path = 'album_melanie.json'
print(resume_index)

with open(file_path, 'w') as file:
    json.dump(album_dict, file)

0
